In [1]:
# Volumes of Optical Labels is calculated. The erosion value serves as the lower error bound. The dilation value serves as the upper error bound.

In [2]:
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from shapely.geometry import box
import os
import numpy as np
import re
import logging
from scipy.ndimage import binary_dilation, binary_erosion, generate_binary_structure

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define structuring elements
dilation_struct_elem = generate_binary_structure(2, 2) # 2D, 8-connectivity for dilation
erosion_struct_elem = generate_binary_structure(2, 2)  # 2D, 4-connectivity for erosion

def calculate_volume_change(row, diff_dem, operation_type):
    """Calculate volume change for dilation, erosion, or normal using a binary operation."""
    out_image, out_transform = mask(diff_dem, [row.geometry], crop=True)
    if out_image.ndim == 3 and out_image.shape[0] == 1:
        out_image = out_image.squeeze(0)  # Remove the first dimension to make it 2D

    valid_mask = (out_image > -500) & (out_image < 8000)

    if operation_type == 'dilation':
        original_mask_count = np.sum(valid_mask)
        mask_used = binary_dilation(valid_mask, structure=dilation_struct_elem)
        dilated_mask_count = np.sum(mask_used)
        original_volume_sum = np.nansum(out_image[valid_mask]) * 100
        dilated_volume_sum = np.nansum(out_image[mask_used]) * 100

    elif operation_type == 'erosion':
        original_mask_count = np.sum(valid_mask)
        mask_used = binary_erosion(valid_mask, structure=erosion_struct_elem)
        eroded_mask_count = np.sum(mask_used)
        original_volume_sum = np.nansum(out_image[valid_mask]) * 100
        eroded_volume_sum = np.nansum(out_image[mask_used]) * 100

    elif operation_type == 'normal':
        mask_used = valid_mask
        normal_mask_count = np.sum(mask_used)
        normal_volume_sum = np.nansum(out_image[mask_used]) * 100

    else:
        raise ValueError(f"Unsupported operation type: {operation_type}")

    return np.nansum(out_image[mask_used]) * 100  # Assuming cell area of 10m x 10m

def find_relevant_dem_file(dem_directory, year, offset):
    """Find the relevant DEM file for a given year with an offset within a directory."""
    target_year = year + offset
    for file in os.listdir(dem_directory):
        match = re.search(r'(\d{4})\.tif$', file)
        if match and int(match.group(1)) == target_year:
            return os.path.join(dem_directory, file)
    return None

def process_geojson(file_path_OT, dem_directory):
    """Process GeoJSON to calculate erosion, dilation, and normal volumes for same, previous, and following years."""
    gdf = gpd.read_file(file_path_OT)
    unique_years = gdf['year'].unique() 

    for year in unique_years:
        for offset, time_suffix in [(-1, 'previous'), (0, 'same'), (1, 'following')]:
            dem_file_path = find_relevant_dem_file(dem_directory, year, offset)
            if not dem_file_path:
                logging.warning(f"No DEM file found for year {year + offset} in directory {dem_directory}")
                continue

            dem = rio.open(dem_file_path)
            dem_polygon = box(*dem.bounds)

            for index, row in gdf[gdf['year'] == year].iterrows():
                if row.geometry.intersects(dem_polygon):
                    n_volume_change = calculate_volume_change(row, diff_dem=dem, operation_type='normal')
                    d_volume_change = calculate_volume_change(row, diff_dem=dem, operation_type='dilation')
                    e_volume_change = calculate_volume_change(row, diff_dem=dem, operation_type='erosion')
                    gdf.at[index, f'normal_{time_suffix}'] = n_volume_change
                    gdf.at[index, f'dilation_{time_suffix}'] = d_volume_change
                    gdf.at[index, f'erosion_{time_suffix}'] = e_volume_change
            dem.close()

    return gdf

# Example usage for processing regions and types
regions = ['herschel', 'kolguev', 'peel', 'gydan']
types = ['OT', 'OP']
for region in regions:
    for type in types:
        output_filepath = f'Results/all/{region}_{type}_labels_results.geojson'
        file_path_OT = f'labels/labels_w_metrics/{region}_{type}_labels_unionized_SimpleMetrics.geojson'
        dem_directory = f'DEM_rasters/DEM_raster_combined/{region}/'
        processed_object = process_geojson(file_path_OT, dem_directory)
        processed_object.to_file(output_filepath, driver='GeoJSON')
        logging.info(f"Processed and saved {region}_{type} results to {output_filepath}")


Normal - Mask count: 106, Volume sum: -46209.04541015625
Dilation - Original mask count: 106, Dilated mask count: 142, Original volume sum: -46209.04541015625, Dilated volume sum: -46209.04846191406
Erosion - Original mask count: 106, Eroded mask count: 62, Original volume sum: -46209.04541015625, Eroded volume sum: -33119.207763671875
Normal - Mask count: 1804, Volume sum: -847073.828125
Dilation - Original mask count: 1804, Dilated mask count: 2063, Original volume sum: -847073.828125, Dilated volume sum: -847073.73046875
Erosion - Original mask count: 1804, Eroded mask count: 1549, Original volume sum: -847073.828125, Eroded volume sum: -727936.767578125
Normal - Mask count: 24, Volume sum: -30.05518913269043
Dilation - Original mask count: 24, Dilated mask count: 51, Original volume sum: -30.05518913269043, Dilated volume sum: -30.05518913269043
Erosion - Original mask count: 24, Eroded mask count: 4, Original volume sum: -30.05518913269043, Eroded volume sum: -74.2344856262207
Nor

2024-05-07 09:43:19,561 - WARNING - No DEM file found for year 2019 in directory DEM_rasters/DEM_raster_combined/herschel/


Normal - Mask count: 106, Volume sum: -41120.50476074219
Dilation - Original mask count: 106, Dilated mask count: 142, Original volume sum: -41120.50476074219, Dilated volume sum: -41120.5078125
Erosion - Original mask count: 106, Eroded mask count: 62, Original volume sum: -41120.50476074219, Eroded volume sum: -30428.952026367188
Normal - Mask count: 1804, Volume sum: -875849.8046875
Dilation - Original mask count: 1804, Dilated mask count: 2063, Original volume sum: -875849.8046875, Dilated volume sum: -875849.8046875
Erosion - Original mask count: 1804, Eroded mask count: 1549, Original volume sum: -875849.8046875, Eroded volume sum: -743901.220703125
Normal - Mask count: 24, Volume sum: 2028.9655685424805
Dilation - Original mask count: 24, Dilated mask count: 51, Original volume sum: 2028.9655685424805, Dilated volume sum: 2028.9653778076172
Erosion - Original mask count: 24, Eroded mask count: 4, Original volume sum: 2028.9655685424805, Eroded volume sum: 197.1266746520996
Norma

2024-05-07 09:43:20,100 - WARNING - No DEM file found for year 2019 in directory DEM_rasters/DEM_raster_combined/herschel/


Dilation - Original mask count: 34, Dilated mask count: 62, Original volume sum: -9461.6455078125, Dilated volume sum: -9461.6455078125
Erosion - Original mask count: 34, Eroded mask count: 10, Original volume sum: -9461.6455078125, Eroded volume sum: -2998.96240234375
Normal - Mask count: 7, Volume sum: -1182.3505401611328
Dilation - Original mask count: 7, Dilated mask count: 15, Original volume sum: -1182.3505401611328, Dilated volume sum: -1182.3505401611328
Erosion - Original mask count: 7, Eroded mask count: 0, Original volume sum: -1182.3505401611328, Eroded volume sum: 0.0
Normal - Mask count: 7, Volume sum: -1372.0916748046875
Dilation - Original mask count: 7, Dilated mask count: 16, Original volume sum: -1372.0916748046875, Dilated volume sum: -1372.0916748046875
Erosion - Original mask count: 7, Eroded mask count: 0, Original volume sum: -1372.0916748046875, Eroded volume sum: 0.0
Normal - Mask count: 40, Volume sum: -4187.213134765625
Dilation - Original mask count: 40, Di

...

2024-05-07 09:43:40,405 - WARNING - No DEM file found for year 2018 in directory DEM_rasters/DEM_raster_combined/gydan/
2024-05-07 09:43:40,409 - WARNING - No DEM file found for year 2019 in directory DEM_rasters/DEM_raster_combined/gydan/


Erosion - Original mask count: 0, Eroded mask count: 0, Original volume sum: 0.0, Eroded volume sum: 0.0
Normal - Mask count: 0, Volume sum: 0.0
Dilation - Original mask count: 0, Dilated mask count: 0, Original volume sum: 0.0, Dilated volume sum: 0.0
Erosion - Original mask count: 0, Eroded mask count: 0, Original volume sum: 0.0, Eroded volume sum: 0.0
Normal - Mask count: 0, Volume sum: 0.0
Dilation - Original mask count: 0, Dilated mask count: 0, Original volume sum: 0.0, Dilated volume sum: 0.0
Erosion - Original mask count: 0, Eroded mask count: 0, Original volume sum: 0.0, Eroded volume sum: 0.0
Normal - Mask count: 12, Volume sum: -2129.4361114501953
Dilation - Original mask count: 12, Dilated mask count: 30, Original volume sum: -2129.4361114501953, Dilated volume sum: -2129.4361114501953
Erosion - Original mask count: 12, Eroded mask count: 0, Original volume sum: -2129.4361114501953, Eroded volume sum: 0.0
Normal - Mask count: 4, Volume sum: 303.8066864013672
Dilation - Ori

2024-05-07 09:43:41,627 - WARNING - No DEM file found for year 2022 in directory DEM_rasters/DEM_raster_combined/gydan/
2024-05-07 09:43:41,629 - WARNING - No DEM file found for year 2023 in directory DEM_rasters/DEM_raster_combined/gydan/


Erosion - Original mask count: 4, Eroded mask count: 0, Original volume sum: -289.32785987854004, Eroded volume sum: 0.0
Normal - Mask count: 33, Volume sum: -6012.847137451172
Dilation - Original mask count: 33, Dilated mask count: 68, Original volume sum: -6012.847137451172, Dilated volume sum: -6012.847137451172
Erosion - Original mask count: 33, Eroded mask count: 0, Original volume sum: -6012.847137451172, Eroded volume sum: 0.0
Normal - Mask count: 28, Volume sum: -3621.950912475586
Dilation - Original mask count: 28, Dilated mask count: 53, Original volume sum: -3621.950912475586, Dilated volume sum: -3621.9512939453125
Erosion - Original mask count: 28, Eroded mask count: 5, Original volume sum: -3621.950912475586, Eroded volume sum: -1234.611415863037
Normal - Mask count: 11, Volume sum: -1931.0396194458008
Dilation - Original mask count: 11, Dilated mask count: 28, Original volume sum: -1931.0396194458008, Dilated volume sum: -1931.0396194458008
Erosion - Original mask count:

2024-05-07 09:43:41,933 - INFO - Processed and saved gydan_OP results to Results/all/gydan_OP_labels_results.geojson
